# Week 1 - Exercise solutions

For all these solutions, we assume you have imported `pandas` and the metabric dataset.

In [3]:
import pandas as pd

metabric = pd.read_csv('metabric_clinical_and_expression_data.csv')

In [2]:
metabric.head()

,Patient_ID,Cohort,Age_at_diagnosis,Survival_time,Survival_status,Vital_status,Chemotherapy,Radiotherapy,Tumour_size,Tumour_stage,...,Integrative_cluster,Mutation_count,ESR1,ERBB2,PGR,TP53,PIK3CA,GATA3,FOXA1,MLPH
0,MB-0000,1,75.65,140.500000,LIVING,Living,NO,YES,22.0,2.0,...,4ER+,NaN,8.929817,9.333972,5.680501,6.338739,5.704157,6.932146,7.953794,9.729728
1,MB-0002,1,43.19,84.633333,LIVING,Living,NO,YES,10.0,1.0,...,4ER+,2.0,10.047059,9.729606,7.505424,6.192507,5.757727,11.251197,11.843989,12.536570
2,MB-0005,1,48.87,163.700000,DECEASED,Died of Disease,YES,NO,15.0,2.0,...,3,2.0,10.041281,9.725825,7.376123,6.404516,6.751566,9.289758,11.698169,10.306115
3,MB-0006,1,47.68,164.933333,LIVING,Living,YES,YES,25.0,2.0,...,9,1.0,10.404685,10.334979,6.815637,6.869241,7.219187,8.667723,11.863379,10.472181
4,MB-0008,1,76.97,41.366667,DECEASED,Died of Disease,YES,YES,40.0,2.0,...,9,2.0,11.276581,9.956267,7.331223,6.337951,5.817818,9.719781,11.625006,12.161961


### Exercises from the Text

**A)** Create a new column that gives the average expression of all the genes with expression data in metabric.

The "obvious" way to do this is rather cumbersome:

In [ ]:
metabric['average_expression']  = (metabric.ESR1 + metabric.ERBB2 + metabric.PGR + metabric.TP53 + metabric.PIK3CA + metabric.GATA3 + metabric.FOXA1 + metabric.MLPH) / 8

A more sophisticated way to do this can be achieved using the subsetting conveniently discussed just after this exercise was posed:

In [7]:
metabric.loc[:,].mean(axis=1)

0       7.575357
1       9.358010
2       8.949169
3       9.080877
4       9.278323
          ...   
1899    9.046193
1900    9.257834
1901    9.171025
1902    9.568569
1903    8.663974
Length: 1904, dtype: float64

In [ ]:
metabric['average_expression'] = metabric.loc[:,'ESR1':'MLPH'].mean(axis=1) # axis=1 ensures the mean is calculated across the columns

In [8]:
metabric.head()

,Patient_ID,Cohort,Age_at_diagnosis,Survival_time,Survival_status,Vital_status,Chemotherapy,Radiotherapy,Tumour_size,Tumour_stage,...,Integrative_cluster,Mutation_count,ESR1,ERBB2,PGR,TP53,PIK3CA,GATA3,FOXA1,MLPH
0,MB-0000,1,75.65,140.500000,LIVING,Living,NO,YES,22.0,2.0,...,4ER+,NaN,8.929817,9.333972,5.680501,6.338739,5.704157,6.932146,7.953794,9.729728
1,MB-0002,1,43.19,84.633333,LIVING,Living,NO,YES,10.0,1.0,...,4ER+,2.0,10.047059,9.729606,7.505424,6.192507,5.757727,11.251197,11.843989,12.536570
2,MB-0005,1,48.87,163.700000,DECEASED,Died of Disease,YES,NO,15.0,2.0,...,3,2.0,10.041281,9.725825,7.376123,6.404516,6.751566,9.289758,11.698169,10.306115
3,MB-0006,1,47.68,164.933333,LIVING,Living,YES,YES,25.0,2.0,...,9,1.0,10.404685,10.334979,6.815637,6.869241,7.219187,8.667723,11.863379,10.472181
4,MB-0008,1,76.97,41.366667,DECEASED,Died of Disease,YES,YES,40.0,2.0,...,9,2.0,11.276581,9.956267,7.331223,6.337951,5.817818,9.719781,11.625006,12.161961


** **

**B)** Write a query to extract data on only those patients who have had both chemotherapy and radiotherapy. Can you compute the average tumour size for such patients? How does this compare to the tumour size for patients who haven't undergone therapy?

Extracting the patients who have undergone chemotherapy and radiotherapy:

In [9]:
metabric[(metabric.Chemotherapy == "YES") & (metabric.Radiotherapy == "YES")]

,Patient_ID,Cohort,Age_at_diagnosis,Survival_time,Survival_status,Vital_status,Chemotherapy,Radiotherapy,Tumour_size,Tumour_stage,...,Integrative_cluster,Mutation_count,ESR1,ERBB2,PGR,TP53,PIK3CA,GATA3,FOXA1,MLPH
3,MB-0006,1,47.68,164.933333,LIVING,Living,YES,YES,25.0,2.0,...,9,1.0,10.404685,10.334979,6.815637,6.869241,7.219187,8.667723,11.863379,10.472181
4,MB-0008,1,76.97,41.366667,DECEASED,Died of Disease,YES,YES,40.0,2.0,...,9,2.0,11.276581,9.956267,7.331223,6.337951,5.817818,9.719781,11.625006,12.161961
6,MB-0014,1,56.45,164.333333,LIVING,Living,YES,YES,10.0,2.0,...,3,4.0,10.793832,9.276507,7.720952,5.992706,7.481835,8.365527,11.482627,10.755199
12,MB-0045,1,45.27,164.900000,LIVING,Living,YES,YES,19.0,2.0,...,4ER-,NaN,6.367051,9.618424,5.828490,6.328115,6.005518,7.360639,7.912785,8.870536
14,MB-0048,1,51.46,103.833333,LIVING,Living,YES,YES,25.0,2.0,...,4ER+,NaN,8.101955,12.847992,6.114007,6.397985,7.779824,7.294771,10.963978,9.063062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876,MB-7267,4,28.37,195.366667,DECEASED,Died of Other Causes,YES,YES,20.5,NaN,...,10,7.0,5.623988,9.413817,5.535405,5.547733,5.571594,6.643888,5.969946,6.427479
1878,MB-7269,4,36.60,21.166667,DECEASED,Died of Disease,YES,YES,25.0,NaN,...,10,4.0,7.121643,11.532133,5.230989,5.768024,6.230033,6.148473,6.905075,6.717019
1880,MB-7273,4,47.47,208.200000,LIVING,Living,YES,YES,20.0,NaN,...,5,5.0,5.815046,13.401624,5.283706,6.647040,5.738257,7.893301,11.119795,11.532189
1887,MB-7281,4,51.22,49.533333,DECEASED,Died of Disease,YES,YES,16.0,NaN,...,5,3.0,6.074817,12.545156,5.218277,6.423540,5.856297,7.084129,11.029630,10.737725


Compute the average tumour size for these patients.

In [11]:
metabric[(metabric.Chemotherapy == "YES") & (metabric.Radiotherapy == "YES")].Tumour_size.mean()

32.320093457943926

An alternative method would be to extract the tumour size column first, and then subset by therapy status:

In [ ]:
metabric.Tumour_size[(metabric.Chemotherapy == "YES") & (metabric.Radiotherapy == "YES")].mean()

Can you think why this approach might be advantageous?

The final part of this exercise can be done via a simple modification to the conditions.

In [12]:
metabric.Tumour_size[(metabric.Chemotherapy == "NO") & (metabric.Radiotherapy == "NO")].mean()

25.582390029325506

** ** 

**C)** Write a query to extract data on patients from cohort 1 with either the highest or second highest tumour stage (part of the exercise is to figure out what the top two tumour stages are!). A method introduced in the final section of this session might be useful...

The first step is to figure out what the two largest tumour stages are. The range of values for tumour stage can be obtained using `.unique()`, from which we see the top two stages are 3 and 4.

In [13]:
metabric.Tumour_stage.unique()

array([ 2.,  1.,  4.,  3., nan,  0.])

Extracting the data is then a relatively simple task.

In [ ]:
metabric[(metabric.Tumour_stage.isin([3, 4])) & (metabric.Cohort==1)]

This can actually be achieved more succintly using the `.nlargest()` method. I wasn't aware of this method until writing this solution, so big congrats to anyone who managed to obtain this solution!

In [ ]:
metabric[(metabric.Tumour_stage.isin( pd.Series(metabric.Tumour_stage.unique()).nlargest(n=2) )) & (metabric.Cohort==1)]

** **

**D)** Use `.sum()` to get a vector of the amount of missing data for each variable.

Recall the `.sum()` method computes the sum of a Series, or the sum for each column when called on a DataFrame.

In [19]:
metabric.isna().sum()

1904

** **

**E)** There is a method that does the opposite of `.isna()` - i.e. it returns `True` when data is present and `False` where there is an `NaN`. Can you find out (or guess) what it is?

The method is called `.notna()`, and works exactly how you would expect it to.

In [ ]:
metabric.notna()

In [ ]:
metabric.isna().sum()

### Exercise 1

- What are the different values of the `Integrative Cluster` variable? 
- Can you produce a series containing the number of patients in each cluster? (There is a useful method that will help you out with this, but I leave it to you to find out what that is).

The different values of the integrative cluster variable can be found using the `.unique()` method.

In [20]:
metabric.Integrative_cluster.unique()

array(['4ER+', '3', '9', '7', '4ER-', '5', '8', '10', '1', '2', '6'],
      dtype=object)

With some googling, you may have found the method `.value_counts()`, that returns the count for each unqie variable in a Series.

In [22]:
metabric.Integrative_cluster.value_counts(normalize=True)

8       0.151786
3       0.148109
4ER+    0.128151
10      0.115021
5       0.096639
7       0.095588
9       0.074580
1       0.069328
6       0.044118
4ER-    0.038866
2       0.037815
Name: Integrative_cluster, dtype: float64

### Exercise 2 

- Read the dataset `metabric_clinical_and_expression_data.csv` and store its summary statistics into a new variable called `metabric_summary`.
- Just like the `.read_csv()` method allows reading data from a file, `pandas` provides a `.to_csv()` method to write `DataFrames` to files. Write your summary statistics object into a file called `metabric_summary.csv`. You can use `help(metabric.to_csv)` to get information on how to use this function.
- Use the help information to modify the previous step so that you can generate a Tab Separated Value (TSV) file instead 
- Similarly, explore the method `to_excel()` to produce an excel spreadsheet containing summary statistics

In [24]:
# Store summary statistics
metabric_summary = metabric.describe(include="all")
metabric_summary

,Patient_ID,Cohort,Age_at_diagnosis,Survival_time,Survival_status,Vital_status,Chemotherapy,Radiotherapy,Tumour_size,Tumour_stage,...,Integrative_cluster,Mutation_count,ESR1,ERBB2,PGR,TP53,PIK3CA,GATA3,FOXA1,MLPH
count,1904,1904.000000,1904.000000,1904.000000,1904,1903,1904,1904,1884.000000,1403.000000,...,1904,1859.000000,1904.000000,1904.000000,1904.000000,1904.000000,1904.000000,1904.000000,1904.000000,1904.000000
unique,1904,NaN,NaN,NaN,2,3,2,2,NaN,NaN,...,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,MB-3046,NaN,NaN,NaN,DECEASED,Living,NO,YES,NaN,NaN,...,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,1103,801,1508,1137,NaN,NaN,...,289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,2.643908,61.087054,125.121324,NaN,NaN,NaN,NaN,26.238726,1.750535,...,NaN,5.697687,9.607824,10.765364,6.237203,6.197967,5.970097,9.502910,10.800526,11.362384
std,NaN,1.228615,12.978711,76.334148,NaN,NaN,NaN,NaN,15.160976,0.628999,...,NaN,4.058778,2.133827,1.357359,1.020871,0.401864,0.352549,1.502636,1.754282,1.687555
min,NaN,1.000000,21.930000,0.000000,NaN,NaN,NaN,NaN,1.000000,0.000000,...,NaN,1.000000,5.217238,6.372949,4.860645,5.201128,5.158697,5.277722,5.184945,5.323652
25%,NaN,1.000000,51.375000,60.825000,NaN,NaN,NaN,NaN,17.000000,1.000000,...,NaN,3.000000,8.092992,9.969681,5.408728,5.930335,5.735007,8.767954,10.829777,11.042871
50%,NaN,3.000000,61.770000,115.616667,NaN,NaN,NaN,NaN,23.000000,2.000000,...,NaN,5.000000,10.252166,10.530301,5.877591,6.185873,5.938094,9.911805,11.367947,11.873967
75%,NaN,3.000000,70.592500,184.716667,NaN,NaN,NaN,NaN,30.000000,2.000000,...,NaN,7.000000,11.268331,11.159306,6.899220,6.456987,6.148720,10.560220,11.779545,12.396317


In [ ]:
help(metabric.to_csv)

This is the basic syntax for writing a data frame to a csv file:

In [26]:
metabric_summary.to_csv("metabric_summary.csv")

The following lines show examples of modifications that can be made to this function:

In [ ]:
metabric_summary.to_csv("metabric_summary.tsv", sep = '\t') # Creates a tsv (tab seperated values)

In [ ]:
metabric_summary.to_csv("metabric_summary.csv", columns = ["Cohort", "Age_at_diagnosis"]) # Selects only certain columns to write

In [ ]:
metabric_summary.to_csv("~/Desktop/metabric_summary.csv", index = False) # More detail on where to write the file to

In [ ]:
# Write an excel spreadsheet

#help(metabric.to_excel)
metabric_summary.to_excel("metabric_summary.xlsx")

#If: ModuleNotFoundError: No module named 'openpyxl'
#pip3 install openpyxl --user

### Exercise 3

- Read the dataset `metabric_clinical_and_expression_data.csv` into a variable e.g. `metabric`.
- Calculate the mean tumour size of patients grouped by vital status and tumour stage
- Find the cohort of patients and tumour stage where the average expression of genes TP53 and FOXA1 is highest
- Do patients with greater tumour size live longer? How about patients with greater tumour stage? How about greater Nottingham_prognostic_index?

This exercise can be done using the `groupby` function.

In [29]:
# Calculate the mean tumour size of patients grouped by vital status and tumour stage
metabric.groupby(['Vital_status', 'Tumour_stage']).mean().Tumour_size

Vital_status          Tumour_stage
Died of Disease       1.0             18.851852
                      2.0             29.604895
                      3.0             48.227273
                      4.0             34.375000
Died of Other Causes  0.0             65.000000
                      1.0             17.924528
                      2.0             28.071429
                      3.0             49.611111
Living                0.0             23.000000
                      1.0             16.588462
                      2.0             25.435535
                      3.0             47.379310
                      4.0             35.000000
Name: Tumour_size, dtype: float64

It's a bit neater to only display the columns we need information on.

In [ ]:
metabric.groupby(['Vital_status', 'Tumour_stage']).mean()['Tumour_size']

In [30]:
# Find the cohort of patients and tumour stage where the average expression of genes TP53 and FOXA1 is highest

metabric.groupby(['Cohort', 'Tumour_stage']).mean()[['TP53', 'FOXA1']]

TP53      FOXA1
Cohort Tumour_stage                     
1      0.0           6.188952  11.673359
       1.0           6.220457  10.691623
       2.0           6.228292  10.785180
       3.0           6.186591  10.510511
       4.0           6.157500  11.599434
2      1.0           6.055063  10.877447
       2.0           6.054182  10.256763
       3.0           5.913632  10.212963
3      1.0           6.217709  11.020944
       2.0           6.213454  10.832707
       3.0           6.203268  10.676141
5      1.0           6.257859  10.964222
       2.0           6.135436  10.696157
       3.0           5.765103  10.794154

To answer problems about survival times, let's restrict ourselves to those patients where we know the survival times.

In [31]:
metabric_deceased = metabric[metabric.Vital_status == 'Died of Disease']

**Note**: This strategy of only looking at already deceased patients does introduce bias, as we are excluding those who survive long enough not to be deceased yet. The question of how best to incoporate survival data on still alive patients (where you only know their survival time is >= what is recorded) is essentially the whole reason *Survival Analysis* is it's own field of statistics.

Tumour Size is a continuous variable, so to answer the question of whether it has an effect on survival times, we are better off looking at correlation.

In [33]:
metabric_deceased.Tumour_size.corr(metabric_deceased.Survival_time, method='spearman')

-0.20938381529171657

For tumour stage, we can return to the group by function we know and love(?).

In [34]:
metabric_deceased.groupby('Tumour_stage').mean()['Survival_time']

Tumour_stage
1.0    104.349383
2.0     76.688112
3.0     48.878109
4.0     51.179167
Name: Survival_time, dtype: float64

When computing grouped averages, it is good practice to display the size of each group.

In [35]:
metabric_deceased.groupby('Tumour_stage').agg(['mean', 'size'])['Survival_time']

,mean,size
Tumour_stage,,
1.0,104.349383,108
2.0,76.688112,286
3.0,48.878109,67
4.0,51.179167,8


Nottingham prognostic index is a continuous variable, so we are back to computing correlations.

In [36]:
metabric_deceased['Nottingham_prognostic_index'].corr(metabric_deceased['Survival_time'])

-0.26785785548328533

### Exercise 4

Review the section on missing data presented in the lecture. Consulting the [user's guide section dedicated to missing data](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html) if necessary use the functionality provided by pandas to answer the following questions:

- Which variables (columns) of the metabric dataset have missing data?
- Find the patients ids who have missing tumour size and/or missing mutation count data. Which cohorts do they belong to?
- For the patients identified to have missing tumour size data for each cohort, calculate the average tumour size of the patients with tumour size data available within the same cohort to fill in the missing data

In [37]:
# Which variables (columns) of the metabric dataset have missing data?
metabric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1904 entries, 0 to 1903
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Patient_ID                     1904 non-null   object 
 1   Cohort                         1904 non-null   int64  
 2   Age_at_diagnosis               1904 non-null   float64
 3   Survival_time                  1904 non-null   float64
 4   Survival_status                1904 non-null   object 
 5   Vital_status                   1903 non-null   object 
 6   Chemotherapy                   1904 non-null   object 
 7   Radiotherapy                   1904 non-null   object 
 8   Tumour_size                    1884 non-null   float64
 9   Tumour_stage                   1403 non-null   float64
 10  Neoplasm_histologic_grade      1832 non-null   float64
 11  Lymph_nodes_examined_positive  1904 non-null   int64  
 12  Lymph_node_status              1904 non-null   i

Find the IDs and cohorts for patients with missing data in either tumour size or mutation count:

In [44]:
metabric.Cohort[(metabric.Tumour_size.isna()) | (metabric.Mutation_count.isna())]

0       1
12      1
14      1
15      1
16      1
       ..
1568    5
1580    5
1609    5
1620    5
1635    5
Name: Cohort, Length: 65, dtype: int64

In [ ]:
metabric[['Patient_ID', 'Cohort']][(metabric.Tumour_size.isna()) | (metabric.Mutation_count.isna())]

It's worth seeing the number of patients in each cohort with missing data, as well as the proportions:

In [45]:
metabric.Cohort[(metabric.Tumour_size.isna()) | (metabric.Mutation_count.isna())].value_counts()

1    46
5    10
3     8
2     1
Name: Cohort, dtype: int64

In [46]:
# normalize=True enable us to compute the proportion of patients within each cohort that are missing data
metabric.Cohort[(metabric.Tumour_size.isna()) | (metabric.Mutation_count.isna())].value_counts(normalize=True)

1    0.707692
5    0.153846
3    0.123077
2    0.015385
Name: Cohort, dtype: float64

In [47]:
metabric.Tumour_size

0       22.0
1       10.0
2       15.0
3       25.0
4       40.0
        ... 
1899    25.0
1900    20.0
1901    25.0
1902    25.0
1903    20.0
Name: Tumour_size, Length: 1904, dtype: float64

For the patients identified to have missing tumour size data for each cohort, 
calculate the average tumour size of the patients with tumour size data available within the same cohort to fill in the missing data

In [48]:
# Compute average tumour sizes for each cohort
avg_tsize_1 = round(metabric.Tumour_size[metabric.Cohort==1].mean(), 1)
avg_tsize_2 = round(metabric.Tumour_size[metabric.Cohort==2].mean(), 1)
avg_tsize_3 = round(metabric.Tumour_size[metabric.Cohort==3].mean(), 1)
avg_tsize_5 = round(metabric.Tumour_size[metabric.Cohort==5].mean(), 1)

# Fill in missing values
metabric[metabric.Cohort==1] = metabric[metabric.Cohort==1].fillna(value={'Tumour_size':avg_tsize_1})
metabric[metabric.Cohort==2] = metabric[metabric.Cohort==2].fillna(value={'Tumour_size':avg_tsize_2})
metabric[metabric.Cohort==3] = metabric[metabric.Cohort==3].fillna(value={'Tumour_size':avg_tsize_3})
metabric[metabric.Cohort==5] = metabric[metabric.Cohort==5].fillna(value={'Tumour_size':avg_tsize_5})

**Bonus Exercise**:

- The above bit of code is begging to be put into a `for` loop. Have a go at this.
- See if you can write a function that takes in a data frame with columns "Cohort" and "Tumour_size", and returns a new data frame with the missing values of tumour size filled in as above.